In [624]:
# Imports all the required packages
import os
import pandas as pd
import argparse
import numpy as np
import os.path as osp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch_geometric.data import (InMemoryDataset, download_url, extract_zip,Data)

import torch
import torch.nn as nn
import torch.nn.functional as F
from rdkit.Chem import DataStructs
from torch_geometric.loader import DataLoader

import torch.nn.functional as F
from torch.nn import Linear
from torch.nn import BatchNorm1d
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_add_pool

try:
    import rdkit
    from rdkit import Chem
    from rdkit.Chem import AllChem
    from rdkit import rdBase
    from rdkit.Chem.rdchem import HybridizationType
    from rdkit import RDConfig
    from rdkit.Chem import ChemicalFeatures
    from rdkit.Chem.rdchem import BondType as BT
    from rdkit.Chem import Draw
    rdBase.DisableLog('rdApp.error')
except ImportError:
    rdkit = None

In [625]:
# Model definition used for regression

class GCNlayer(nn.Module):
    
    def __init__(self, n_features, conv_dim1, conv_dim2, conv_dim3, concat_dim, dropout):
        super(GCNlayer, self).__init__()
        self.n_features = n_features
        self.conv_dim1 = conv_dim1
        self.conv_dim2 = conv_dim2
        self.conv_dim3 = conv_dim3
        self.concat_dim =  concat_dim
        self.dropout = dropout   # Dropout Layer 
        
        self.conv1 = GCNConv(self.n_features, self.conv_dim1)
        self.bn1 = BatchNorm1d(self.conv_dim1)
        self.conv2 = GCNConv(self.conv_dim1, self.conv_dim2)
        self.bn2 = BatchNorm1d(self.conv_dim2)
        self.conv3 = GCNConv(self.conv_dim2, self.conv_dim3)
        self.bn3 = BatchNorm1d(self.conv_dim3)
        self.conv4 = GCNConv(self.conv_dim3, self.concat_dim)
        self.bn4 = BatchNorm1d(self.concat_dim)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.bn3(x)
        x = F.relu(self.conv4(x, edge_index))
        x = self.bn4(x)
        
        # Global sum aggregrator function
        x = global_add_pool(x, data.batch) 
        x = F.dropout(x, p=self.dropout, training=self.training)
        return x
    
class FClayer(nn.Module):
    
    def __init__(self, concat_dim, pred_dim1, pred_dim2, out_dim, dropout):
        super(FClayer, self).__init__()
        self.concat_dim = concat_dim
        self.pred_dim1 = pred_dim1
        self.pred_dim2 = pred_dim2
        self.out_dim = out_dim
        self.dropout = dropout

        self.fc1 = Linear(self.concat_dim, self.pred_dim1)
        self.bn1 = BatchNorm1d(self.pred_dim1)
        self.fc2 = Linear(self.pred_dim1, self.pred_dim2)
        self.fc3 = Linear(self.pred_dim2, self.out_dim)
    
    def forward(self, data):
        x = F.elu(self.fc1(data))
        x = self.bn1(x)
        x = F.elu(self.fc2(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc3(x)
        return x

# Model definition  
class GCN_Model_reg(nn.Module):
    def __init__(self, args):
        super(GCN_Model_reg, self).__init__()
        
        # Convolutional Layer call
        self.conv = GCNlayer(args.n_features,args.conv_dim1,args.conv_dim2,args.conv_dim3,args.concat_dim,args.dropout)

        # Fully connected layer call
        self.fc = FClayer(args.concat_dim,args.pred_dim1,args.pred_dim2,args.out_dim,args.dropout)
        
    def forward(self, data):
        x = self.conv(data) # Calling the convolutional layer
        x = self.fc(x) # Calling the Fully Connected Layer
        return x

In [626]:
# pytorch will be run on cpu
seed=200
paser = argparse.ArgumentParser()
args = paser.parse_args("")
np.random.seed(200)
torch.manual_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [627]:
# Inputs required by the program
args.epoch = 400
args.lr = 0.0001
args.optim = 'Adam'
args.step_size = 20
args.gamma = 0.9
args.dropout = 0.1
args.n_features = 29
dim = 512
args.conv_dim1 = dim
args.conv_dim2 = dim
args.conv_dim3 = dim
args.concat_dim = dim
args.pred_dim1 = 64
args.pred_dim2 = 32
args.out_dim = 1

In [628]:
# encoding of all variables

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))
 
def get_intervals(l):
    """For list of lists, gets the cumulative products of the lengths"""
    intervals = len(l) * [0]
    # Initalize with 1
    intervals[0] = 1
    for k in range(1, len(l)):
        intervals[k] = (len(l[k]) + 1) * intervals[k - 1]
    return intervals

def safe_index(l, e):
    """Gets the index of e in l, providing an index of len(l) if not found"""
    try:
        return l.index(e)
    except:
        print("Add value to list of length {}".format(len(l)))
        return len(l)

In [629]:
# Atom features

possible_atom_list = ['H','C', 'O', 'F','N','Cl','P','S','Si','Br','I']  # Atomic symbol 11

aromatic=[0,1] # Aromatic 1
isring=[0,1]   # Ring 1
possible_numH_list = [0, 1, 2, 3, 4] # Total Number of bonded hydrogen atoms possible 5
num_bonds = [0, 1, 2, 3, 4, 5]  # Total Number of Hs a carbon can bond / Total number of bonds an atom make 6
possible_formal_charge_list = [-4,-3, -2, -1, 0, 1, 2, 3, 4]

# sp3d is removed because it doesnot vary according to the paper . Hybridization 4
possible_hybridization_list = [Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,Chem.rdchem.HybridizationType.SP3,Chem.rdchem.HybridizationType.SP3D2]

# storing all the features in a detailed list
reference_lists = [possible_atom_list, possible_numH_list,possible_formal_charge_list, num_bonds,aromatic,isring,possible_hybridization_list]

intervals = get_intervals(reference_lists)
print(intervals)

[1, 6, 60, 420, 1260, 3780, 18900]


- Total number of atom features used here is = 29

In [630]:
# Concatenate the entire feature list
def get_feature_list(atom):
    features = 5 * [0]
    features[0] = safe_index(possible_atom_list, atom.GetSymbol())
    features[1] = safe_index(possible_numH_list, atom.GetTotalNumHs())
    features[2] = safe_index(num_bonds, atom.GetImplicitValence())
    features[3] = safe_index(possible_formal_charge_list, atom.GetFormalCharge())    
    features[4] = safe_index(possible_hybridization_list, atom.GetHybridization())
    return features

In [631]:
def features_to_id(features, intervals):
    """Convert list of features into index using spacings provided in intervals"""
    id = 0
    for k in range(len(intervals)):
        id += features[k] * intervals[k]
    id = id + 1
    return id

def id_to_features(id, intervals):
    features = 6 * [0]
    id -= 1
    for k in range(0, 6 - 1):
        features[6 - k - 1] = id // intervals[6 - k - 1]
        id -= features[6 - k - 1] * intervals[6 - k - 1]
    features[0] = id
    return features

def atom_to_id(atom):
    """Return a unique id corresponding to the atom type"""
    features = get_feature_list(atom)
    return features_to_id(features, intervals)


In [632]:
def atom_features(atom,bool_id_feat=False,explicit_H=False):
    if bool_id_feat:
        return np.array([atom_to_id(atom)])
    else:
        from rdkit import Chem
        results = np.array(one_of_k_encoding_unk(atom.GetSymbol(),possible_atom_list) + 
                           one_of_k_encoding_unk(atom.GetImplicitValence(), num_bonds) + 
                           [atom.GetFormalCharge()] + 
                           one_of_k_encoding_unk(atom.GetHybridization(), possible_hybridization_list) + 
                           [atom.GetIsAromatic()]+[atom.IsInRing()])
    if not explicit_H:
        results = np.array(results.tolist() + one_of_k_encoding_unk(atom.GetTotalNumHs(),possible_numH_list))
    return np.array(results)

In [633]:
# Bond Features

def bond_features(bond):
    from rdkit import Chem
    bt = bond.GetBondType()
    bond_feats = [bt == Chem.rdchem.BondType.SINGLE, bt == Chem.rdchem.BondType.DOUBLE,
                  bt == Chem.rdchem.BondType.TRIPLE, bt == Chem.rdchem.BondType.AROMATIC,
                  bond.GetIsConjugated(),bond.IsInRing()]
    
    # Include stereo bond features as it effects the boiling point
    bond_feats = bond_feats + one_of_k_encoding_unk(str(bond.GetStereo()),["STEREONONE", "STEREOANY", "STEREOZ", "STEREOE"])
    return np.array(bond_feats)


In [634]:
# Create  atom pair of two connected molecules to pass messages

def get_bond_pair(mol):
    bonds = mol.GetBonds()
    res = [[],[]]
    for bond in bonds:
        res[0] += [bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()]
        res[1] += [bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()]
    return res

# create a graph data structure comprising of x=node_f, edge_index = bond pair info, 
def mol2vec(mol):
    atoms = mol.GetAtoms()
    bonds = mol.GetBonds()
    node_f= [atom_features(atom) for atom in atoms]
    edge_index = get_bond_pair(mol)
    edge_attr = [bond_features(bond) for bond in bonds]

    for bond in bonds:
        edge_attr.append(bond_features(bond))
        
    # Graph data to be used
    data = Data(x=torch.tensor(node_f, dtype=torch.float),edge_index=torch.tensor(edge_index, dtype=torch.long),edge_attr=torch.tensor(edge_attr,dtype=torch.float))
    
    return data

In [635]:
# Return mol info
def make_mol(df):
    mols = {}
    for i in range(df.shape[0]):
        mols[Chem.MolFromSmiles(df['Smiles'].iloc[i])] = df['Tb'].iloc[i]
    return mols

In [636]:
# Convert the dataset with X values and y values to be trained/tested
def make_vec(mols):
    X = [mol2vec(m) for m in mols.keys()]
    for i, data in enumerate(X):
        y = list(mols.values())[i]
        data.y = torch.tensor([y], dtype=torch.float)
    return X

In [637]:
# Importing Dataset
dataset = pd.read_csv('../data/raw_data.csv', low_memory=False)
dataset = pd.concat([dataset['Smiles'], dataset['Tb']], axis=1)
dataset

,Smiles,Tb
0,CCC(O)CCC(F)(F)F,413.80
1,CCC(CC)ON(=O)=O,413.15
2,Clc1ccc(Cl)c(c1)C(=O)O,574.15
3,NC(C)Cc1ccccc1,476.15
4,COc1ccc(cc1)NC(=O)C,608.15
...,...,...
5271,BrC(F)(F)Oc1ccc(cc1)C(C)(C)COCc2cccc(Oc3ccccc3)c2,564.15
5272,OCC(C)CC(F)(F)F,405.65
5273,Clc1ccc(cc1)CC2CCC(C)(C)C2(O)CN3C=NC=N3,558.15
5274,CCOC(=O)C(Cl)Cc1cc(c(F)cc1Cl)N2N=C(C)N(C(F)F)C2=O,625.65


In [638]:

# Standardize the Boiling Point values
mean=dataset['Tb'].mean()
std=dataset['Tb'].std()
dataset['Tb']=(dataset['Tb']-mean)/std
dataset.describe()



,Tb
count,5.276000e+03
mean,-8.080471e-17
std,1.000000e+00
min,-4.212019e+00
25%,-6.301239e-01
50%,-7.437000e-03
75%,6.041305e-01
max,2.303198e+01


In [639]:
# Save Checkpoint for model evaluation :
def save_checkpoint(epoch, model, optimizer, filename):
    state = {'Epoch': epoch,'State_dict': model.state_dict(),'optimizer': optimizer.state_dict()}
    torch.save(state, filename)

In [640]:
# Train test split 
Train_set, Test_set = train_test_split(dataset, test_size=0.3, shuffle=True, random_state=seed)
Train_set = Train_set.reset_index(drop=True)
Test_set = Test_set.reset_index(drop=True)
Train_set

,Smiles,Tb
0,CCOC(=O)c1ccc(Cl)cc1,0.520735
1,CC1CCCC(C)(C)C1=O,-0.135310
2,c1ccc2Sc3ccccc3Sc2c1,1.938459
3,N#CSCCCC,-0.051915
4,CCCCCCCCCCC(=O)OC,0.637489
...,...,...
3688,O=COC,-1.767639
3689,CCCCCCCCCCCCCCCCCCCCCCCC,2.230900
3690,Brc1ccc(C)cc1,-0.070818
3691,ClN,3.661968


In [641]:
train_mols = make_mol(Train_set)
test_mols = make_mol(Test_set)

train_X = make_vec(train_mols)
test_X = make_vec(test_mols)


In [642]:
# Model activation

model = GCN_Model_reg(args)
model = model.to(device)

In [643]:
train_loader = DataLoader(train_X, batch_size=100, shuffle=True, drop_last=True)
test_loader = DataLoader(test_X, batch_size=len(test_X), shuffle=False, drop_last=True)

In [644]:
def train(model, device, optimizer, train_loader, criterion, args):
    train_correct = 0
    train_total = 0
    epoch_train_loss = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        targets = data.y.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        
        #print(outputs)
        #print(targets)
        outputs.require_grad = False
        _, predicted = torch.max(outputs.data, 1)
        train_total += targets.size(0)

        correct_dim_output=outputs.squeeze() # done to reshape the outputs and match with targets
        loss = criterion(correct_dim_output, targets)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    epoch_train_loss /= len(train_loader)
    print('- Loss : %.4f' % epoch_train_loss)
    return model, epoch_train_loss

In [645]:
def experiment(model, train_loader, device, args):

    
    optimizer = optim.Adam(model.parameters(),lr=args.lr)
    criterion = nn.MSELoss()  # decide which loss to be used later
    scheduler = optim.lr_scheduler.StepLR(optimizer,
                                          step_size=args.step_size,
                                          gamma=args.gamma)
    
    list_train_loss = list()
    list_train_acc = list()
    print('[Train]')
    for epoch in range(args.epoch):
        scheduler.step()
        print('- Epoch :', epoch+1)
        model, train_loss= train(model, device, optimizer, train_loader, criterion, args)
        list_train_loss.append(train_loss)
    
    
    
     
    save_checkpoint(epoch, model, optimizer, 'model.pt')
    
    return args

In [646]:
result = vars(experiment(model, train_loader, device, args))

[Train]
- Epoch : 1


c:\Users\kanad\Desktop\Github repos\ME_793_PROJECT\793_env\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


- Loss : 0.5784
- Epoch : 2
- Loss : 0.4039
- Epoch : 3
- Loss : 0.3547
- Epoch : 4
- Loss : 0.3124
- Epoch : 5
- Loss : 0.2933
- Epoch : 6
- Loss : 0.2760
- Epoch : 7
- Loss : 0.2715
- Epoch : 8
- Loss : 0.2590
- Epoch : 9
- Loss : 0.2514
- Epoch : 10
- Loss : 0.2502
- Epoch : 11
- Loss : 0.2434
- Epoch : 12
- Loss : 0.2300
- Epoch : 13
- Loss : 0.2237
- Epoch : 14
- Loss : 0.2229
- Epoch : 15
- Loss : 0.2228
- Epoch : 16
- Loss : 0.2102
- Epoch : 17
- Loss : 0.1995
- Epoch : 18
- Loss : 0.2058
- Epoch : 19
- Loss : 0.1943
- Epoch : 20
- Loss : 0.2034
- Epoch : 21
- Loss : 0.1050
- Epoch : 22
- Loss : 0.1831
- Epoch : 23
- Loss : 0.1805
- Epoch : 24
- Loss : 0.1996
- Epoch : 25
- Loss : 0.2030
- Epoch : 26
- Loss : 0.1811
- Epoch : 27
- Loss : 0.1759
- Epoch : 28
- Loss : 0.1800
- Epoch : 29
- Loss : 0.1870
- Epoch : 30
- Loss : 0.1768
- Epoch : 31
- Loss : 0.1720
- Epoch : 32
- Loss : 0.1651
- Epoch : 33
- Loss : 0.1840
- Epoch : 34
- Loss : 0.1788
- Epoch : 35
- Loss : 0.1605
- Epoc

KeyboardInterrupt: 